# The diffusion equation - parallelized

In [ ]:
# Note, this is only for jupyter notebooks
import ipyparallel as ipp
# Start the cluster
cluster = ipp.Cluster(engines="mpi", n=8)
client = cluster.start_and_connect_sync()
client.ids

In [ ]:
%px %matplotlib inline

In [ ]:
%%px
import numpy as np
import matplotlib.pyplot as plt
from mpi4py import MPI
#
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

print('Rank {}/{} is alive.'.format(rank, size))

#### Define the spatial grid and the time increment

In [ ]:
%%px
nx = 1000
dx = 0.1
nt = 10000
dt = 0.001
D = 1 # diffusion constant

#### Domain decomposition

In [ ]:
%%px
# Domain decomposition: set up domain boundaries
nx1 = rank*nx//size
nx2 = (rank+1)*nx//size

print('{}, Domain boundaries: {}-{}'.format(rank, nx1, nx2-1))

# We include one additional cell at the boundaries for communication purposes
x = np.arange(nx1-1, nx2+1)*dx

#### Initial condition: We use a Gaussian concentration profile initially

In [ ]:
%%px
sigma0 = 20*dx
c = np.exp(-(x-nx*dx/2)**2/(2*sigma0**2)) / (np.sqrt(2*np.pi)*sigma0)

plt.title('rank ${}$'.format(rank))
plt.xlim(0, nx*dx)
plt.ylim(0, 0.3)
plt.plot(x, c)

#### Propagate in time

In [ ]:
%%px
for t in range(nt):
    # Send to right, receive from left
    comm.Sendrecv(c[-2:-1], (rank+1)%size, recvbuf=c[:1], source=(rank-1)%size)
    # Send to left, receive from right
    comm.Sendrecv(c[1:2], (rank-1)%size, recvbuf=c[-1:], source=(rank+1)%size)
    if t % (nt/5) == 0:
        plt.plot(x, c, '-', label='$t={}$'.format(t*dt))

    d2c_dx2 = (np.roll(c, 1)-2*c+np.roll(c, -1))/(dx**2)
    c += D*d2c_dx2*dt
plt.legend(loc='best')

In [ ]:
%%px
print(c.shape)

#### Gather all data on rank 0 and plot

In [ ]:
%%px
x_full_range = np.arange(nx)*dx
c_full_range = np.zeros(nx)
comm.Gather(c[1:-1], c_full_range, root=0)
if rank == 0:
    plt.plot(x_full_range, c_full_range, '-')

#### Gather during progation so we can plot the evolution of the concentration profile

In [ ]:
%%px
if rank == 0:
    x_full_range = np.arange(nx)*dx
    c_full_range = np.zeros(nx)
for t in range(nt):
    # Send to right, receive from left
    comm.Sendrecv(c[-2:-1], (rank+1)%size, recvbuf=c[:1], source=(rank-1)%size)
    # Send to left, receive from right
    comm.Sendrecv(c[1:2], (rank-1)%size, recvbuf=c[-1:], source=(rank+1)%size)
    if t % (nt/5) == 0:
        comm.Gather(c[1:-1], c_full_range, root=0)
        if rank == 0:
            plt.plot(x_full_range, c_full_range, '-', label='$t={}$'.format(t*dt))

    d2c_dx2 = (np.roll(c, 1)-2*c+np.roll(c, -1))/(dx**2)
    c += D*d2c_dx2*dt
if rank == 0:
    plt.legend(loc='best')

In [ ]:
%%px
if rank == 1: print(c_full_range)

In [ ]:
%%px
print('Process {} knows about dx = {}'.format(rank,dx))

In [ ]:
%%px 
c_full_range